In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import requests
import csv
from io import BytesIO
import pandas as pd

#先呼叫chromdedriver
main_driver = webdriver.Chrome('C:\\Users\\user\\Desktop\\Conda\\chromedriver')  # 注意你們放CHROMEDRIVER的位置
#告訴chromedriver 等下要找的element 如果沒有找到，要等10秒讓他們生完
main_driver.implicitly_wait(20)
#打開目標網頁
main_driver.get('https://fred.stlouisfed.org/series/DRSFRMACBS')

download_btn_ele = main_driver.find_element_by_xpath('//*[@id="download-button"]')
download_btn_ele.click()

excel_download_btn_ele = main_driver.find_element_by_xpath('//*[@id="fg-download-menu"]/li[1]/a')

head={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
#輸入第9行的網址就會直接下載到最新的cfnai的excel檔案
url = excel_download_btn_ele.get_attribute('href')
#因為這邊下載的直接就是excel檔案，用pandas套件來寫會更簡單
#不需要像tutorial_cfnai_1檔案中11-13行先download=requests.get(url, headers=head) 再 df =  pd.read_excel(BytesIO(download.content))
#直接用read_excel(url地址)就可以了
df = pd.read_excel(url)
df.head(20)

,FRED Graph Observations,Unnamed: 1
0,Federal Reserve Economic Data,NaN
1,Link: https://fred.stlouisfed.org,NaN
2,Help: https://fred.stlouisfed.org/help-faq,NaN
3,Economic Research Division,NaN
4,Federal Reserve Bank of St. Louis,NaN
5,NaN,NaN
6,DRSFRMACBS,Delinquency Rate on Single-Family Residential ...
7,NaN,NaN
8,"Frequency: Quarterly, End of Period",NaN
9,observation_date,DRSFRMACBS


In [3]:
#把index 0-9的資料drop(丟)掉
df = df.drop(df.index[[0,1,2,3,4,5,6,7,8,9]])
#顯示d出前5項row
df.head()

,FRED Graph Observations,Unnamed: 1
10,1991-01-01 00:00:00,3.1
11,1991-04-01 00:00:00,3.16
12,1991-07-01 00:00:00,3.23
13,1991-10-01 00:00:00,3.3
14,1992-01-01 00:00:00,3.13


In [4]:
#重新安排index，雖然index整理好了但會發現時間居然有道小時-分-秒 但我們只要年-月-日而已
df = df.reset_index(drop=True)
df.head()

,FRED Graph Observations,Unnamed: 1
0,1991-01-01 00:00:00,3.1
1,1991-04-01 00:00:00,3.16
2,1991-07-01 00:00:00,3.23
3,1991-10-01 00:00:00,3.3
4,1992-01-01 00:00:00,3.13


In [5]:
#將時間格從新整理我們要的年-月-日格式
df['FRED Graph Observations'] = pd.to_datetime(df['FRED Graph Observations'], format="%Y%m%d")
df.head()

,FRED Graph Observations,Unnamed: 1
0,1991-01-01,3.1
1,1991-04-01,3.16
2,1991-07-01,3.23
3,1991-10-01,3.3
4,1992-01-01,3.13


In [6]:
#將FRED Graph Observations欄位的名稱改為Date
df = df.rename(columns = {'FRED Graph Observations':'Date'})
#將Unnamed: 1欄位的名稱改為Value
df = df.rename(columns = {'Unnamed: 1':'Value'})
#這樣就完成了!
df.head(20)

,Date,Value
0,1991-01-01,3.1
1,1991-04-01,3.16
2,1991-07-01,3.23
3,1991-10-01,3.3
4,1992-01-01,3.13
5,1992-04-01,3.04
6,1992-07-01,2.89
7,1992-10-01,2.79
8,1993-01-01,2.79
9,1993-04-01,2.66
